In [38]:
t2 = [('A', 7), ('A', 9), ('A', 2), ('B', 4), ('B', 3)]

In [39]:
rdd=spark.sparkContext.parallelize(t2)

In [40]:
rdd.collect()

[('A', 7), ('A', 9), ('A', 2), ('B', 4), ('B', 3)]

# 1. Find average per key using reduceByKey()

In [136]:
rdd2=rdd.mapValues(lambda x: (x,1))

In [137]:
rdd2.collect()

[('A', (7, 1)), ('A', (9, 1)), ('A', (2, 1)), ('B', (4, 1)), ('B', (3, 1))]

In [138]:
red=rdd2.reduceByKey(lambda x, y : (x[0]+y[0], x[1] + y[1]) )

In [139]:
red.collect()

[('A', (18, 3)), ('B', (7, 2))]

In [140]:
final=red.mapValues(lambda x: x[0]/ x[1])

In [141]:
final.collect()

[('A', 6.0), ('B', 3.5)]

# 2. Find average per key using groupByKey()

In [314]:
rdd3=rdd.groupByKey()

In [316]:
rdd3.collect()

[('A', <pyspark.resultiterable.ResultIterable at 0x7febf7fd3ad0>),
 ('B', <pyspark.resultiterable.ResultIterable at 0x7febf7fd3d50>)]

In [143]:
final2=rdd3.mapValues(lambda x: sum(x)/len(x))

In [144]:
final2.collect()

[('A', 6.0), ('B', 3.5)]

 Given RDD[(String, Integer)], find median per key.  
Provide two solutions: using groupByKey() and reduceByKey()

# 3.  Using groupByKey() 

In [164]:
pairs= rdd.map(lambda x: (x[0], int(x[1])))

In [165]:
pairs.collect()

[('A', 7), ('A', 9), ('A', 2), ('B', 4), ('B', 3)]

In [166]:
def find_median(x):
    array_list=x[1]
    med=np.median(array_list)
    return (x[0], med)


In [167]:
array_key=pairs.groupByKey().mapValues(lambda iter: list(iter)).sortByKey()

In [168]:
array_key.collect()

[('A', [7, 9, 2]), ('B', [4, 3])]

In [169]:
key_med=array_key.map(lambda rec: find_median(rec))

In [ ]:
key_med.collect()

# 4.  Given RDD[(String, Integer)], find average of positive numbers per key.  
##Provide two solutions: using groupByKey() and reduceByKey()

In [162]:
t3 = [('A', 7), ('A', 9), ('A', 2), ('B', 4), ('B', 3), ('B', -4), ('A', -2)]

In [163]:
rdd_new=spark.sparkContext.parallelize(t3)

In [164]:
rdd_new.collect()

[('A', 7), ('A', 9), ('A', 2), ('B', 4), ('B', 3), ('B', -4), ('A', -2)]

# ReduceByKey

In [165]:
rdd_f=rdd_new.filter(lambda x: (x[1] > 0))

In [166]:
rdd_f.collect()

[('A', 7), ('A', 9), ('A', 2), ('B', 4), ('B', 3)]

In [168]:
rdd_m = rdd_f.mapValues(lambda x: (x,1))

In [169]:
rdd_m.collect()

[('A', (7, 1)), ('A', (9, 1)), ('A', (2, 1)), ('B', (4, 1)), ('B', (3, 1))]

In [170]:
rdd_r=rdd_m.reduceByKey(lambda x,y:(x[0]+y[0], x[1]+ y[1]))

In [171]:
rdd_r.collect()

[('A', (18, 3)), ('B', (7, 2))]

In [172]:
rdd_final = rdd_r.mapValues(lambda x: x[0]/x[1])

In [173]:
rdd_final.collect()

[('A', 6.0), ('B', 3.5)]

# GroupByKey

In [174]:
rdd_2=rdd_new.filter(lambda x: (x[1] > 0))

In [175]:
rdd_2.collect()

[('A', 7), ('A', 9), ('A', 2), ('B', 4), ('B', 3)]

In [176]:
rdd3=rdd_2.groupByKey()

In [178]:
rdd_fin=rdd3.mapValues(lambda x: sum(x)/len(x))

In [179]:
rdd_fin.collect()

[('A', 6.0), ('B', 3.5)]

# 5. Given RDD[(String, Integer)], find (minimum, maximum) per key.  
Provide two solutions: using groupByKey() and reduceByKey()

In [99]:
t4=[('A', 2),('A', 4), ('A', 3), ('B', 2), ('B', 3) ]

In [100]:
rd=spark.sparkContext.parallelize(t4)

In [101]:
rd.collect()

[('A', 2), ('A', 4), ('A', 3), ('B', 2), ('B', 3)]

In [102]:
pairs = rd.map(lambda x: (x[0], int(x[1]))).sortByKey()

In [104]:
pairs.collect()

[('A', 2), ('A', 4), ('A', 3), ('B', 2), ('B', 3)]

In [105]:
pairs_rdd = pairs.map(lambda x: (x[0], (int(x[1]), int(x[1]))))

In [106]:
pairs_rdd.collect()

[('A', (2, 2)), ('A', (4, 4)), ('A', (3, 3)), ('B', (2, 2)), ('B', (3, 3))]

In [107]:
min_max = pairs_rdd.reduceByKey(lambda x,y:(min(x[0],y[0]), max(x[1],y[1]))).sortByKey()

In [108]:
min_max.collect()

[('A', (2, 4)), ('B', (2, 3))]

===================================================

#  6. Find min and max of list

In [56]:
ty=[2, 4, 6, -2, 0 , 90]

In [57]:
rddu=spark.sparkContext.parallelize(ty)

In [58]:
def find_min_max(Partition): 
    first_time=False
    for v in Partition: 
        if (first_time==False):
            min2=v
            max2=v
            first_time=True
        else: 
            min2=min(min2, v)
            max2=max(max2, v)
    return [(min2, max2)]

In [59]:
min_max=rddu.mapPartitions(find_min_max)

In [60]:
final_min_max = min_max.reduce(lambda x,y: (min(x[0], y[0]), max(x[1], y[1])))

In [61]:
final_min_max

(-2, 90)

# 7. Find count, min and max of list

In [135]:
ty=[2, 4, 6, -2, 0 , 90]

In [136]:
rdd_n=spark.sparkContext.parallelize(ty)

In [137]:
def find_count_min_max(Partition): 
    first_time=False
    counter=0
    for v in Partition: 
        if (first_time==False):
            min2=v
            max2=v
            first_time=True
            counter+=1
        else: 
            min2=min(min2, v)
            max2=max(max2, v)
            counter+=1
    return [(counter, min2, max2)]

In [138]:
cou_min_max=rdd_n.mapPartitions(find_count_min_max)

In [139]:
cou_min_max.collect()

[(1, 2, 2), (2, 4, 6), (1, -2, -2), (2, 0, 90)]

In [140]:
final = cou_min_max.reduce(lambda x, y : ((x[0] + y[0]), min(x[1], y[1]), max(x[2], y[2])))

In [141]:
final

(6, -2, 90)

=================================================

# 8. Find min, max, number of +ves, number of -ves of list

In [148]:
ty=[2, 4, 6, -2, 0 , 90]

In [149]:
rdd_ne=spark.sparkContext.parallelize(ty)

In [150]:
def find_posi_min_max(Partition): 
    first_time=False
    posi = 0
    nega = 0
    for v in Partition: 
        if (first_time==False):
            min2=v
            max2=v
            first_time=True
            if (v<0):
                nega+=1
            else:
                posi+=1
        else: 
            min2=min(min2, v)
            max2=max(max2, v)
            if (v<0):
                nega+=1
            else:
                posi+=1
    return [( min2, max2, posi, nega)]

In [151]:
posi_min_max=rdd_ne.mapPartitions(find_posi_min_max)

In [152]:
posi_min_max.collect()

[(2, 2, 1, 0), (4, 6, 2, 0), (-2, -2, 0, 1), (0, 90, 2, 0)]

In [162]:
final_ans = posi_min_max.reduce(lambda x, y : ( min(x[0], y[0]), max(x[1], y[1]), (x[2] + y[2]), (x[3] + y[3])))

In [163]:
final_ans

(-2, 90, 5, 1)

===============================================================

# 9 Write a series of PySpark transformations to find (average age of kids, average-number-of-candies)

<age, no of candies>

In [216]:
data = [(2,4), (2,3), (3,4), (4,5), (4,1), (5,4), (5,10), (5,5), (6, 0), (6,7)]

In [217]:
rdd9 = spark.sparkContext.parallelize(data)

In [218]:
rdd9.collect()

[(2, 4),
 (2, 3),
 (3, 4),
 (4, 5),
 (4, 1),
 (5, 4),
 (5, 10),
 (5, 5),
 (6, 0),
 (6, 7)]

In [219]:
rdd10 = rdd9.map(lambda x: (x[0], 1 , x[1], 1))

In [220]:
rdd10.collect()

[(2, 1, 4, 1),
 (2, 1, 3, 1),
 (3, 1, 4, 1),
 (4, 1, 5, 1),
 (4, 1, 1, 1),
 (5, 1, 4, 1),
 (5, 1, 10, 1),
 (5, 1, 5, 1),
 (6, 1, 0, 1),
 (6, 1, 7, 1)]

In [221]:
rdd11 = rdd10.reduce(lambda x,y: ((x[0] + y[0]), (x[1] + y[1]), (x[2] + y[2]), (x[3] + y[3])))

In [222]:
rdd11

(42, 10, 43, 10)

In [226]:
rdd12 =(rdd11[0]/rdd11[1], rdd11[2]/rdd11[3])

In [227]:
rdd12

(4.2, 4.3)

===========================================

In [303]:
def find_sum_elements(Partitions): 
    sum1=0
    sum2=0
    count=0
    for val in Partitions:
        sum1+=val[0]
        sum2+=val[1]
        count+=1
    return[(sum1, sum2, count)]
            
    

In [304]:
rdd9.collect()

[(2, 4),
 (2, 3),
 (3, 4),
 (4, 5),
 (4, 1),
 (5, 4),
 (5, 10),
 (5, 5),
 (6, 0),
 (6, 7)]

In [305]:
rdd13=rdd9.mapPartitions(find_sum_elements)

In [306]:
rdd13.collect()

[(4, 7, 2), (7, 9, 2), (9, 5, 2), (22, 22, 4)]

In [309]:
rdd_final = rdd13.reduce(lambda x,y: (x[0]+ y[0], x[1] + y[1], x[2]+ y[2]))

In [310]:
rdd_final

(42, 43, 10)

In [312]:
final = (rdd_final[0]/ rdd_final[2], rdd_final[1]/rdd_final[2])

In [313]:
final

(4.2, 4.3)

====================================================================================

# INverted index

In [ ]:
rdd = spark.sparkContext.textFile(input_path)
rdd.collect()

rdd1 = rdd.map(lambda a: a.split(":"))
rdd1.collect()

def find_labels(text):
    l = []
    words = []
    words = text[1].split(" ")
    count = 1
    for word in words:
        l.append((word, (text[0], count)))
        count += 1
    return (l) 

rdd2 = rdd1.map(lambda a: find_labels(a))
rdd2.collect()

rdd3 = rdd2.flatMap(lambda x: x).groupByKey().mapValues(lambda iter: list(iter))
rdd3.collect()

def map_labels(i):
    d = {}
    for item in i:
        if item[0] in d:
            d[item[0]].append(item[1])
        else:
            d[item[0]] = [item[1]]
    return (d)      

invert = rdd3.mapValues(lambda i: map_labels(i)).sortByKey()
invert.collect()